In [ ]:
#%pip install pandas numpy matplotlib seaborn statsmodels scikit-learn patsy tensorflow keras pytorch

In [ ]:
"""Brief examples of time series operations"""
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

# Generate sample time series data
np.random.seed(42)
dates = pd.date_range('2023-01-01', periods=365, freq='D')
trend = np.linspace(0, 10, 365)
seasonal = 5 * np.sin(2 * np.pi * np.arange(365) / 30)  # 30-day seasonality
noise = np.random.normal(0, 1, 365)
data = trend + seasonal + noise

# Create time series
ts = pd.Series(data, index=dates)

# DateTime operations
print("\n=== DateTime Operations ===")
ts_ny = pd.Timestamp('2023-01-01').tz_localize('America/New_York')
print(f"NY Time: {ts_ny}")
print(f"UTC Time: {ts_ny.tz_convert('UTC')}")

# Resampling
print("\n=== Resampling ===")
monthly_mean = ts.resample('ME').mean()
print("Monthly averages:", monthly_mean.head())

# Rolling windows
print("\n=== Rolling Windows ===")
rolling_mean = ts.rolling(window=7).mean()
print("7-day rolling average:", rolling_mean.head())

# Time series decomposition
print("\n=== Decomposition ===")
decomposition = seasonal_decompose(ts, period=30)
components = {
    'Trend': decomposition.trend.iloc[0],
    'Seasonal': decomposition.seasonal.iloc[0],
    'Residual': decomposition.resid.iloc[0]
}
print("Components:", components)

# Optional: Plot components
import matplotlib.pyplot as plt
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 10))
ts.plot(ax=ax1, title='Original')
decomposition.trend.plot(ax=ax2, title='Trend')
decomposition.seasonal.plot(ax=ax3, title='Seasonal')
decomposition.resid.plot(ax=ax4, title='Residual')
plt.tight_layout()
plt.show()


In [ ]:
"""Brief examples of statsmodels usage"""
import numpy as np
import pandas as pd
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

# Generate sample data
np.random.seed(42)
X = np.random.randn(100, 3)  # 3 features
y = 2 * X[:, 0] + 3 * X[:, 1] - X[:, 2] + np.random.randn(100)

# Linear regression
print("\n=== Linear Regression ===")
X_with_const = sm.add_constant(X)
model = OLS(y, X_with_const)  # Using direct OLS import
results = model.fit()
print(results.summary().tables[1])  # Coefficient table

# GLM example
print("\n=== GLM ===")
# Binary outcome for logistic regression
y_binary = (y > y.mean()).astype(int)
glm_model = sm.GLM(y_binary, X_with_const, family=sm.families.Binomial())
glm_results = glm_model.fit()
print("GLM coefficients:", glm_results.params)

# Panel data example
print("\n=== Panel Data ===")
# Generate panel data
entities = 10
times = 5
panel_idx = pd.MultiIndex.from_product([range(entities), range(times)],
                                     names=['entity', 'time'])
panel_data = pd.DataFrame({
    'y': np.random.randn(entities * times),
    'x1': np.random.randn(entities * times)
}, index=panel_idx)

# Removing panel regression temporarily as PanelOLS location needs verification
print("Panel regression example temporarily removed")

# Robust regression
print("\n=== Robust Regression ===")
rlm_model = sm.RLM(y, X_with_const)
rlm_results = rlm_model.fit()
print("Robust regression coefficients:", rlm_results.params)

# Optional: Diagnostic plots
import matplotlib.pyplot as plt
fig = sm.graphics.plot_regress_exog(results, 'x1')
plt.show()


In [ ]:
"""Brief examples of scikit-learn usage"""
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

# Generate sample data
np.random.seed(42)
X = np.random.randn(1000, 20)  # 20 features
y = (X[:, 0] + X[:, 1] > 0).astype(int)  # Binary classification
X_cat = np.random.choice(['A', 'B', 'C'], size=(1000, 2))  # Categorical features

# Preprocessing
print("\n=== Preprocessing ===")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaled data mean:", X_scaled.mean(axis=0)[:3])

enc = OneHotEncoder(sparse_output=False)
X_encoded = enc.fit_transform(X_cat)
print("Encoded categories:", enc.categories_)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Classification example
print("\n=== Classification ===")
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Cross-validation
print("\n=== Cross-validation ===")
scores = cross_val_score(clf, X_scaled, y, cv=5)
print("CV scores:", scores)

# Pipeline example
print("\n=== Pipeline ===")
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
])
pipe.fit(X, y)
print("Pipeline score:", pipe.score(X, y))

# Dimensionality reduction
print("\n=== PCA ===")
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)
print("Explained variance ratio:", pca.explained_variance_ratio_)

# Optional: Plot PCA results
import matplotlib.pyplot as plt
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y)
plt.title('PCA visualization')
plt.show()


In [ ]:
"""Brief examples of Patsy usage"""
import numpy as np
import pandas as pd
import patsy
import statsmodels.api as sm

# Generate sample data
np.random.seed(42)
data = pd.DataFrame({
    'y': np.random.randn(100),
    'x1': np.random.randn(100),
    'x2': np.random.randn(100),
    'category': np.random.choice(['A', 'B', 'C'], 100)
})

# Basic formula
print("\n=== Basic Formula ===")
y, X = patsy.dmatrices('y ~ x1 + x2', data)
print("Design matrix shape:", X.shape)

# Transformations
print("\n=== Transformations ===")
y, X = patsy.dmatrices('y ~ standardize(x1) + center(x2)', data)
print("Transformed x1 mean:", X[:, 1].mean())
print("Transformed x2 mean:", X[:, 2].mean())

# Categorical variables
print("\n=== Categorical Variables ===")
y, X = patsy.dmatrices('y ~ C(category)', data)
print("Category design matrix:\n", X)

# Interactions
print("\n=== Interactions ===")
y, X = patsy.dmatrices('y ~ x1 + x2 + x1:x2', data)
print("Interaction design matrix shape:", X.shape)

# Custom transformations
print("\n=== Custom Transformations ===")
def log_plus_1(x):
    return np.log(x - x.min() + 1)

y, X = patsy.dmatrices('y ~ log_plus_1(x1)', data)
print("Custom transformation result:", X[:, 1][:5])

# Integration with statsmodels
print("\n=== Statsmodels Integration ===")
model = sm.OLS.from_formula('y ~ x1 + x2', data)
results = model.fit()
print(results.params)


In [ ]:
"""Brief examples of Keras usage"""
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Generate sample data
np.random.seed(42)
X = np.random.randn(1000, 28, 28, 1)  # Simulated MNIST-like data
y = np.random.randint(0, 10, 1000)  # 10 classes

# Sequential API example
def create_sequential_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Functional API example
def create_functional_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(10, activation='softmax')(x)
    return keras.Model(inputs, outputs)

# Custom model example
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.pool = keras.layers.MaxPooling2D(2)
        self.flatten = keras.layers.Flatten()
        self.dense = keras.layers.Dense(10, activation='softmax')
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool(x)
        x = self.flatten(x)
        return self.dense(x)

# Data preprocessing
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


# Train sequential model
model = create_sequential_model()
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Add callbacks
callbacks = [
    keras.callbacks.EarlyStopping(patience=3),
    keras.callbacks.ModelCheckpoint('best_model.h5'),
    keras.callbacks.TensorBoard(log_dir='./logs')
]

# Train with data augmentation
model.fit(datagen.flow(X, y, batch_size=32),
            epochs=5,
            callbacks=callbacks)

# Save and load model
model.save('sequential_model.h5')
loaded_model = keras.models.load_model('sequential_model.h5')


In [ ]:
"""Brief examples of PyTorch usage"""
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Generate sample data
np.random.seed(42)
X = torch.FloatTensor(np.random.randn(1000, 1, 28, 28))  # Simulated MNIST-like data
y = torch.LongTensor(np.random.randint(0, 10, 1000))  # 10 classes

# Custom dataset
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create data loader
dataset = SimpleDataset(X, y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define model
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(64 * 7 * 7, 10)
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# Training loop
def train_model(model, loader, epochs=2):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % 10 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

if __name__ == '__main__':
    model = SimpleCNN()
    train_model(model, loader)
    
    # Save model
    torch.save(model.state_dict(), 'simple_cnn.pt')
    
    # Load model
    new_model = SimpleCNN()
    new_model.load_state_dict(torch.load('simple_cnn.pt'))
